In [1]:
from europa import EuropaModule, EuropaConfig
from europa.datasets import BIDProtoDataModule

/home/rafael/miniconda3/envs/europa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-05 12:33:56.479089: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 12:33:56.532920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 12:33:57.553625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Coul

In [2]:
import os

# This can be passed as an environment variable or in 
# the config file
os.environ["HF_TOKEN"] = "YOUR-HF-TOKEN"

In [3]:
config = EuropaConfig.from_yaml("../config/simple_config.yaml")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/paligemma-3b-pt-224.
401 Client Error. (Request ID: Root=1-66b0aa98-693af9240a1186010297538e;0d506d05-4b93-484e-acd5-c8c2358dadef)

Cannot access gated repo for url https://huggingface.co/google/paligemma-3b-pt-224/resolve/main/processor_config.json.
Access to model google/paligemma-3b-pt-224 is restricted. You must be authenticated to access it.

In [ ]:
datamodule = BIDProtoDataModule(config.data)

In [3]:
module = EuropaModule(config)

Unused kwargs: ['bnb_4bit_compute_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.63s/it]


In [5]:
import lightning as L

trainer = L.Trainer(
        accelerator="gpu",
        devices=[0],
        max_epochs=config.train.max_epochs,
        accumulate_grad_batches=config.train.accumulate_grad_batches,
        check_val_every_n_epoch=config.train.check_val_every_n_epoch,
        gradient_clip_val=config.train.gradient_clip_val,
        precision="16-mixed",
        limit_val_batches=5,
        num_sanity_val_steps=0,
        # logger=wandb_logger,
)

Using 16bit Automatic Mixed Precision (AMP)
/home/rafael/miniconda3/envs/europa/lib/python3.10/site-packages/lightning/pytorch/plugins/precision/amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(module, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | PeftModelForCausalLM | 1.7 B  | train
-------------------------------------------------------
11.3 M    Trainable params
1.7 B     Non-trainable params
1.7 B     Total params
6,948.584 Total estimated model params size (MB)


Epoch 0:   0%|          | 0/128 [00:00<?, ?it/s] 

/home/rafael/miniconda3/envs/europa/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:430: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 10.44 MiB is free. Including non-PyTorch memory, this process has 3.80 GiB memory in use. Of the allocated memory 3.62 GiB is allocated by PyTorch, and 105.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)